In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import random
import Levenshtein
import collections
from gensim.models import Word2Vec
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
hindi_array = []
english_array = []
with open("hindi_english_transliteration.txt") as f:
    for line in f:
        key, val = line.split()
        hindi_array.append(key)

with open("20k.txt") as f:
    for line in f:
        key = line.split()
        english_array.append(key[0])        

In [3]:
random.shuffle(hindi_array)
random.shuffle(english_array)
Train_Data_Hindi = hindi_array[:18000]
Train_Data_English = english_array[:18000]
Test_Data_Hindi = hindi_array[28815:]
Test_Data_English = english_array[18000:]


In [4]:
count_hindi = {}
count_english = {}
for data in Train_Data_Hindi:
    if data not in count_hindi.keys():
        count_hindi[data] = 0
    count_hindi[data]+=1
for data in Train_Data_English:
    if data not in count_english.keys():
        count_english[data] = 0
    count_english[data]+=1

In [5]:
del count_english['hai']

In [6]:
Data_Training = dict()
for data in Train_Data_English:
    if data not in Data_Training.keys():
        Data_Training[data] = 1
for data in Train_Data_Hindi:
    if data not in Data_Training.keys():
        Data_Training[data] = 0
Data_Testing = Test_Data_English +  Test_Data_Hindi

In [7]:
def KNN(Data_Training , Data_Testing):
    y_pred = []
    for data in Data_Testing:
        data_train = Data_Training.keys()
        data_train = list(data_train)
        random.shuffle(data_train)
        l = len(data)
        min_edit_dict = dict()
        if data in count_english.keys():
            y_pred.append(1)
        elif data in count_hindi.keys():
            y_pred.append(0)
        else:
            for k in data_train:
                set_result = set.intersection(set(data), set(k))
                if len(set_result) > l/2 and abs(len(k)-len(data))/max(len(k),len(data)) < 0.6:
                    if k not in min_edit_dict.keys():
                        min_edit_dict[Levenshtein.distance(str(data),str(k))] = Data_Training[k]
            od = collections.OrderedDict(sorted(min_edit_dict.items()))
            od = list(od.values())
            y_pred.append(1) if od[:5].count(1)>od[:5].count(0) else y_pred.append(0)
    return y_pred

In [8]:
y_pred = KNN(Data_Training , Data_Testing)
(y_pred[:len(Test_Data_English)].count(1) + y_pred[len(Test_Data_English):].count(0))/len(Data_Testing) 


0.86075

In [9]:
KNN(Data_Training , ['ok' , 'then' ,  'talk' , 'to' , 'you' , 'later' , 'abhi' , 'thoda' , 'kam' , 'hai'])

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

In [10]:
SVM_Train_Array = []
SVM_label = []
SVM_Train_List = []
with open("Hindi_English_FB.txt") as f:
    for line in f:
        if line == '\n':
            SVM_Train_List.append(SVM_Train_Array)
            SVM_Train_Array = []
        else:
            key , label , encod = line.split()
            if label == 'en' or label == 'hi':
                SVM_Train_Array.append(key.lower())
                if label == 'en':
                    SVM_label.append(1)
                else:
                    SVM_label.append(0)
SVM_Train_List.append(SVM_Train_Array)
                    
SVM_Train_Array = []
with open("Hindi_english.txt") as f:
    for line in f:
        if line == '\n':
            SVM_Train_List.append(SVM_Train_Array)
            SVM_Train_Array = []
        else:
            key , label , encod = line.split()
            if label == 'en' or label == 'hi':
                SVM_Train_Array.append(key.lower())
                if label == 'en':
                    SVM_label.append(1)
                else:
                    SVM_label.append(0)

SVM_Train_Array = []
with open("Hindi_English_Twitter.txt") as f:
    for line in f:
        if line == '\n':
            SVM_Train_List.append(SVM_Train_Array)
            SVM_Train_Array = []
        else:
            key , label , encod = line.split()
            if label == 'en' or label == 'hi':
                SVM_Train_Array.append(key.lower())
                if label == 'en':
                    SVM_label.append(1)
                else:
                    SVM_label.append(0)                

In [11]:
SVM_Training_model = Word2Vec(SVM_Train_List , window = 5,size = 100,min_count=1)

In [12]:
SVM_Training = []
SVM_Testing = []
for i in SVM_Train_List[:2500]:
    for j in i:
        SVM_Training.append(SVM_Training_model[j])
for i in SVM_Train_List[2500:]:
    for j in i:
        SVM_Testing.append(SVM_Training_model[j])
        

/home/lovish/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/home/lovish/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [13]:
def SVM(SVM_Training , SVM_label,SVM_Testing):
    clf = LinearSVC(C = 5)
    clf.fit(SVM_Training,SVM_label[:30532])
    y_pred = clf.predict(SVM_Testing)
    check = SVM_label[30532:]
    count = 0
    for j in range(len(y_pred)):
        if y_pred[j] == check[j]:
            count+=1
    return count/len(check)

In [14]:
SVM(SVM_Training,SVM_label,SVM_Testing)

0.6811819595645412

In [15]:
def Random_Forest(SVM_Training , SVM_label,SVM_Testing):
    clf1  = RandomForestClassifier(n_estimators=206, random_state=16)
    clf1.fit(SVM_Training,SVM_label[:30532])
    y_pred1 = clf1.predict(SVM_Testing)
    count = 0
    check = SVM_label[30532:]
    for j in range(len(y_pred1)):
        if y_pred1[j] == check[j]:
            count+=1
    return count/len(check)

In [16]:
Random_Forest(SVM_Training , SVM_label,SVM_Testing)

0.8491446345256609